!kaggle datasets list -s "MPST: Movie Plot Synopses with Tags"

!kaggle datasets download -d "cryptexcode/mpst-movie-plot-synopses-with-tags"

from zipfile import ZipFile
with ZipFile("mpst-movie-plot-synopses-with-tags.zip",'r') as zipfile:
    zipfile.extractall("data")

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


.\movie-recommender.ipynb
.\mpst-movie-plot-synopses-with-tags.zip
.\.idea\.gitignore
.\.idea\content based recommender.iml
.\.idea\misc.xml
.\.idea\modules.xml
.\.idea\workspace.xml
.\.idea\inspectionProfiles\profiles_settings.xml
.\.idea\inspectionProfiles\Project_Default.xml
.\.ipynb_checkpoints\movie-recommender-checkpoint.ipynb
.\data\mpst_full_data.csv
.\data\partition.json


In [2]:
data = pd.read_csv('data/mpst_full_data.csv')

In [3]:
data = data.drop_duplicates(subset='title').reset_index(drop=True)

In [4]:
print(data.shape,'\n')
print(data.head(),'\n\n')
print(data.describe())

(13757, 6) 

     imdb_id                                          title  \
0  tt0057603                        I tre volti della paura   
1  tt1733125  Dungeons & Dragons: The Book of Vile Darkness   
2  tt0033045                     The Shop Around the Corner   
3  tt0113862                             Mr. Holland's Opus   
4  tt0086250                                       Scarface   

                                       plot_synopsis  \
0  Note: this synopsis is for the orginal Italian...   
1  Two thousand years ago, Nhagruul the Foul, a s...   
2  Matuschek's, a gift store in Budapest, is the ...   
3  Glenn Holland, not a morning person by anyone'...   
4  In May 1980, a Cuban man named Tony Montana (A...   

                                                tags  split synopsis_source  
0          cult, horror, gothic, murder, atmospheric  train            imdb  
1                                           violence  train            imdb  
2                                    

In [5]:
ind_dict = dict(zip(data['title'],data.index))

In [6]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


import nltk
nltk.download("stopwords")
nltk.download('punkt')

In [7]:
class CustomTokenizer:
    def __init__(self):
        self.stemmer = PorterStemmer()
    def __call__(self, doc):
        return [self.stemmer.stem(token) for token in word_tokenize(doc) if token.isalpha()]
custom_tokenizer = CustomTokenizer()
stopwords = custom_tokenizer(' '.join(stopwords.words('english')))

In [ ]:
stopwords

In [8]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vect = TfidfVectorizer(stop_words=stopwords,max_features=500,token_pattern=r'\b[A-Za-z][A-Za-z]+\b', tokenizer=custom_tokenizer)
bag = vect.fit_transform(data['plot_synopsis']).toarray()
movie_df = pd.DataFrame(bag,columns=vect.get_feature_names_out())

c:\python39\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['becau'] not in stop_words.
  warnings.warn(


In [ ]:
lk = linear_kernel(bag,bag)

In [ ]:
def recommend_movie(movie_name):
    idx = ind_dict['The Shawshank Redemption']
    tags = data.loc[idx,'tags'].split(', ')
    lk_w_idx = list(enumerate(lk[idx]))
    found = [tupl[0] for tupl in sorted(lk_w_idx,reverse=True, key=lambda x:x[1])]
    movies  = data.loc[found,['title']][1:100]
    movies['score'] = range(99,0,-1)
    for idx in movies.index.to_list():
    movie_tags = data.loc[idx, 'tags'].split(", ")
    for tag in movie_tags:
        if tag in tags:
            movies.loc[idx,'score'] += 10
        else:
            movies.loc[idx,'score'] -= 2
    return movies.sort_values(by='score',ascending=False)[:10]